In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm

In [2]:
# Sample asset returns data
np.random.seed(42)
num_obs = 1000
num_assets = 5  # Adjust this to the number of assets in your portfolio

# Generating sample returns for multiple assets
returns = np.random.normal(0.001, 0.02, size=(num_obs, num_assets))

# Creating a DataFrame with named columns
columns = [f"Asset{i}" for i in range(1, num_assets + 1)]
returns_data = pd.DataFrame(returns, columns=columns)

In [8]:
import numpy as np
import pymc3 as pm

# Sample asset returns data
np.random.seed(42)
num_obs = 1000
num_assets = 5  # Adjust this to the number of assets in your portfolio

# Generating sample returns for multiple assets
returns = np.random.normal(0.001, 0.02, size=(num_obs, num_assets))

# Define the model for portfolio volatility
with pm.Model() as portfolio_model:
    # Prior distributions for asset weights
    w = pm.Dirichlet('weights', a=np.ones(num_assets))
    
    # Expected portfolio return (can be adjusted based on your assumptions)
    #mu = pm.Normal('mu', mu=0, sigma=0.01)
    mu = pm.Normal('mu', mu=-0.0017, sigma=0.0082)
    portfolio_return = pm.Deterministic('portfolio_return', pm.math.dot(w, [mu] * num_assets))
    
    # Covariance matrix (prior distribution)
    cov_matrix = pm.Lognormal('cov_matrix', mu=np.log(0.01), sigma=0.5, shape=(num_assets, num_assets))
    
    # Portfolio volatility (standard deviation)
    portfolio_volatility = pm.Deterministic('portfolio_volatility', pm.math.sqrt(pm.math.dot(w, pm.math.dot(cov_matrix, w.T))))
    
    # Likelihood (observed data)
    returns_observed = pm.Normal('returns_observed', mu=portfolio_return, sigma=portfolio_volatility, observed=returns)
    
    # Sampling
    trace = pm.sample(2000, tune=1000)
    
    # Forecasting 10 days of volatility using posterior predictive sampling
    future_returns = np.random.normal(0.001, 0.02, size=(10, num_assets))  # Simulated future returns
    future_data = {returns_observed: future_returns}
    
    # Performing posterior predictive sampling for future volatility
    posterior_pred = pm.sample_posterior_predictive(trace, var_names=['portfolio_volatility'], samples=1, model=portfolio_model)
    
    # Extracting and printing forecasted volatility for the next 10 days
    future_volatility = posterior_pred['portfolio_volatility'][-1]
    print("Forecasted volatility for the next 10 days:")
    print(future_volatility)


/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [cov_matrix, mu, weights]


/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 20 seconds.
There were 846 divergences after tuning. Increase `target_accept` or reparameterize.
There were 788 divergences after tu

Forecasted volatility for the next 10 days:
0.02019046174545063


In [7]:
np.ones(num_assets)

array([1., 1., 1., 1., 1.])

In [6]:
returns

array([[ 1.09342831e-02, -1.76528602e-03,  1.39537708e-02,
         3.14605971e-02, -3.68306749e-03],
       [-3.68273914e-03,  3.25842563e-02,  1.63486946e-02,
        -8.38948772e-03,  1.18512009e-02],
       [-8.26835386e-03, -8.31459507e-03,  5.83924543e-03,
        -3.72656049e-02, -3.34983567e-02],
       ...,
       [ 3.37159561e-02, -3.42084927e-03,  2.38739605e-03,
         4.85193837e-03,  4.88421927e-02],
       [-4.09871253e-02,  1.46644603e-02, -1.29604498e-03,
         1.23354345e-02, -1.21474511e-02],
       [ 2.06993828e-05,  1.52282116e-02,  6.32582040e-02,
         1.71607238e-02, -1.59613120e-02]])

In [4]:
# Summary statistics of posterior distributions
summary = pm.summary(trace)
print(summary)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


                       mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
mu                    0.001  0.000   0.001    0.002      0.000    0.000   
weights[0]            0.250  0.430   0.000    0.997      0.215    0.164   
weights[1]            0.249  0.430   0.000    0.997      0.215    0.164   
weights[2]            0.250  0.430   0.000    0.997      0.215    0.165   
weights[3]            0.001  0.001   0.000    0.004      0.000    0.000   
weights[4]            0.250  0.431   0.000    0.997      0.215    0.165   
portfolio_return      0.001  0.000   0.001    0.002      0.000    0.000   
cov_matrix[0, 0]      0.009  0.007   0.000    0.022      0.003    0.002   
cov_matrix[0, 1]      0.011  0.005   0.003    0.020      0.000    0.000   
cov_matrix[0, 2]      0.010  0.006   0.002    0.020      0.000    0.000   
cov_matrix[0, 3]      0.011  0.006   0.003    0.021      0.000    0.000   
cov_matrix[0, 4]      0.011  0.005   0.003    0.020      0.000    0.000   
cov_matrix[1, 0]      0.0

In [5]:
summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.001,0.000,0.001,0.002,0.000,0.000,752.0,753.0,1.01
weights[0],0.250,0.430,0.000,0.997,0.215,0.164,7.0,27.0,1.56
weights[1],0.249,0.430,0.000,0.997,0.215,0.164,7.0,25.0,1.61
weights[2],0.250,0.430,0.000,0.997,0.215,0.165,7.0,30.0,1.60
weights[3],0.001,0.001,0.000,0.004,0.000,0.000,208.0,152.0,1.03
weights[4],0.250,0.431,0.000,0.997,0.215,0.165,7.0,31.0,1.54
portfolio_return,0.001,0.000,0.001,0.002,0.000,0.000,752.0,753.0,1.01
"cov_matrix[0, 0]",0.009,0.007,0.000,0.022,0.003,0.002,7.0,34.0,1.56
"cov_matrix[0, 1]",0.011,0.005,0.003,0.020,0.000,0.000,109.0,1824.0,1.03
"cov_matrix[0, 2]",0.010,0.006,0.002,0.020,0.000,0.000,169.0,169.0,1.03
